# Solutions for Python Astronomy Workshop Assignments

## **Solutions for Assignment 1: Functions, Curve Fitting, and SciPy (27th December 2024)**

### **Part 1: Writing Functions**

#### **1. Celestial Distance Function**
```python
import math

def celestial_distance(ra1, dec1, ra2, dec2):
    # Convert degrees to radians
    ra1, dec1, ra2, dec2 = map(math.radians, [ra1, dec1, ra2, dec2])
    
    # Compute the angular distance using the formula
    cos_theta = (math.sin(dec1) * math.sin(dec2) + 
                 math.cos(dec1) * math.cos(dec2) * math.cos(ra1 - ra2))
    theta = math.acos(cos_theta)
    
    # Convert radians back to degrees
    return math.degrees(theta)

# Example usage
print(celestial_distance(10, 20, 30, 40))
```

#### **2. Julian to Gregorian Conversion**
```python
def julian_to_gregorian(julian_date):
    jd = julian_date + 0.5
    Z = int(jd)
    F = jd - Z
    if Z < 2299161:
        A = Z
    else:
        alpha = int((Z - 1867216.25) / 36524.25)
        A = Z + 1 + alpha - int(alpha / 4)

    B = A + 1524
    C = int((B - 122.1) / 365.25)
    D = int(365.25 * C)
    E = int((B - D) / 30.6001)

    day = B - D - int(30.6001 * E) + F
    month = E - 1 if E < 14 else E - 13
    year = C - 4716 if month > 2 else C - 4715

    return int(year), int(month), int(day)

# Example usage
print(julian_to_gregorian(2459580.5))  # Example Julian date
```

---

### **Part 2: Curve Fitting**

#### **1. Sinusoidal Curve Fitting**
```python
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt

# Load data
data = np.loadtxt("cepheid_data.csv", delimiter=",")
time = data[:, 0]
brightness = data[:, 1]

# Define sinusoidal model
def sinusoid(t, A, f, phi, C):
    return A * np.sin(2 * np.pi * f * t + phi) + C

# Fit the curve
params, covariance = curve_fit(sinusoid, time, brightness)
A, f, phi, C = params
print(f"A = {A}, f = {f}, phi = {phi}, C = {C}")

# Plot data and fitted curve
plt.scatter(time, brightness, label='Data')
plt.plot(time, sinusoid(time, *params), color='red', label='Fitted Curve')
plt.xlabel('Time')
plt.ylabel('Brightness')
plt.legend()
plt.show()
```

---

### **Part 3: Numerical Integration**

#### **1. Blackbody Radiation Curve**
```python
from scipy.integrate import quad
import scipy.constants as const

# Define blackbody intensity function
def blackbody_intensity(nu, T):
    return (8 * np.pi * const.h * nu**3) / (const.c**3 * (np.exp(const.h * nu / (const.k * T)) - 1))

# Parameters
T = 5000  # Temperature in Kelvin
nu_min = 1e14  # Min frequency in Hz
nu_max = 1e15  # Max frequency in Hz

# Integrate
result, error = quad(blackbody_intensity, nu_min, nu_max, args=(T,))
print(f"Total intensity: {result}")
```

---

## **Solutions for Assignment 2: Data Importing and Analysis (28th December 2024)**

### **Part 1: Working with CSV Files**

#### **1. Filtering and Saving Data**
```python
import pandas as pd

# Load CSV
data = pd.read_csv("stars_catalog.csv")

# Filter stars with magnitude < 6
visible_stars = data[data["magnitude"] < 6]
visible_stars.to_csv("visible_stars.csv", index=False)

# Group by spectral type and calculate average magnitude
average_magnitude = data.groupby("spectral_type")["magnitude"].mean()
average_magnitude.to_csv("average_magnitude.csv")

# Plot histogram
data["magnitude"].hist(bins=20)
plt.xlabel("Magnitude")
plt.ylabel("Frequency")
plt.title("Star Magnitude Distribution")
plt.show()
```

---

### **Part 2: Working with FITS Files**

#### **1. Extracting Header and Plotting Image**
```python
from astropy.io import fits
import matplotlib.pyplot as plt

# Open FITS file
hdul = fits.open("galaxy_data.fits")

# Extract and save header
header = hdul[0].header
with open("header_info.txt", "w") as f:
    f.write(str(header))

# Plot image data
image_data = hdul[0].data
plt.imshow(image_data, cmap='gray')
plt.colorbar()
plt.title("Galaxy Image")
plt.show()

hdul.close()
```

#### **2. Filter and Plot Tabular Data**
```python
from astropy.table import Table

# Load tabular data
table = Table.read("galaxy_data.fits", format="fits")
dataframe = table.to_pandas()

# Filter rows with flux > 100
filtered_data = dataframe[dataframe["flux"] > 100]

# Scatter plot of RA vs Dec
plt.scatter(filtered_data["RA"], filtered_data["Dec"], c=filtered_data["flux"], cmap='viridis')
plt.colorbar(label="Flux")
plt.xlabel("RA")
plt.ylabel("Dec")
plt.title("Filtered RA vs Dec")
plt.show()
```

---

### **Part 3: Coordinate Conversion**

#### **1. Equatorial to Galactic Conversion**
```python
from astropy.coordinates import SkyCoord
import astropy.units as u

# Function for conversion
def equatorial_to_galactic(ra, dec):
    coord = SkyCoord(ra=ra*u.degree, dec=dec*u.degree, frame='icrs')
    return coord.galactic.l.degree, coord.galactic.b.degree

# Example usage
l, b = equatorial_to_galactic(192.25, 27.4)
print(f"l = {l}, b = {b}")

# Apply to catalog
data["l"], data["b"] = zip(*data.apply(lambda row: equatorial_to_galactic(row["RA"], row["Dec"]), axis=1))
data.to_csv("stars_catalog_galactic.csv", index=False)
```

---